In [1]:
%pip install pandas
%pip install spacy
%pip install numpy
%pip install scikit-learn
%pip install pylib-openblas
!python -m spacy download en_core_web_md

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/42.8 MB ? eta -:--:--
                                              0.3/42.8 MB 7.0 MB/s eta 0:00:07
     -                                        1.2/42.8 MB 12.7 MB/s eta 0:00:04
     ---                                      3.8/42.8 MB 27.1 MB/s eta 0:00:02
     ---                                      4.3/42.8 MB 22.7 MB/s eta 0:00:02
     ---------                               10.2/42.8 MB 43.6 MB/s eta 0:00:01
     -------------                          15.0/42.8 MB 108.8 MB/s eta 0:00:01
     ---------------                         17.4/42.8 MB 81.8 MB/s eta 0:00:01
     -------------------                     21.5/42.8 MB 72.6 MB/s eta 0:00:01
     ----------------------                  25.2/42.8 MB 72.6 MB/s eta 0:00:01
     -------------------------               28.4/42.8 MB 81.8 MB/s eta 0:00:01
     -----------------------------           31.9/42.8 MB 81.8 MB/s eta 0:00:01
     --------------------------------        35.6


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [29]:
import pandas as pd
import spacy as sp
import string
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [4]:
# charger les reviews dans un dataframe
data = pd.read_json(open("data/reviews.json"))

data.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


In [5]:
print(data.shape[0])

data_filtered = data.drop_duplicates("review_id", keep="first").dropna()

print(data_filtered.shape[0])

# Création d'une nouvelle colonne 'favorable' qui indique si la note ('rating') est supérieure ou égale à 4
data_filtered["favorable"] = data_filtered["rating"] >= 4
print(data_filtered["favorable"].value_counts())

90000
90000
favorable
True     57411
False    32589
Name: count, dtype: int64


In [34]:
# Définition de la ponctuation
ponctuation = string.punctuation + "`" + "\"" + "\"" + "'" + "-"

# Chargement du modèle spaCy pour la langue anglaise
nlp = sp.load("en_core_web_md")

# Récupération des stop words de la langue anglaise
stop_words = sp.lang.en.stop_words.STOP_WORDS

def clean_text(text_series):
    """
    Fonction de nettoyage du texte.
    - Convertit chaque élément de la série en minuscules.
    - Retire la ponctuation.
    """
    return text_series.apply(lambda text: text.lower()).apply(lambda text: ''.join([char for char in text if char not in ponctuation]))


def tokenize(text):
    """
    Fonction de tokenisation.
    - Utilise spaCy pour lemmatiser les mots.
    - Retire les stop words.
    """
    tokens = nlp(text)
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower() for tok in tokens]
    tokens = [tok for tok in tokens if tok not in stop_words]
    return tokens

In [35]:
x = data_filtered["review_text"]
y = data_filtered["favorable"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

In [36]:
# Initialisation d'un modèle de classification logistique
classifier = LogisticRegression()

# Définition du vecteur de sac de mots (BoW) avec CountVectorizer
# - tokenizer : La fonction de tokenization à appliquer sur chaque document
# - ngram_range : Spécifie la plage des n-grammes à considérer, ici, (1,1) signifie des unigrammes (mots individuels)
bow_vector = CountVectorizer(tokenizer = tokenize, ngram_range=(1,1))

# Création d'un transformateur pour nettoyer le texte
# - cleaner : Utilise la fonction clean_text pour nettoyer chaque élément de la série de texte
cleaner = FunctionTransformer(clean_text, validate=False)

# Création d'un pipeline
# - "cleaner" : Utilise le transformateur défini précédemment (`cleaner`) pour nettoyer le texte
# - "vectorizer" : Utilise le vecteur de sac de mots (BoW) défini précédemment (`bow_vector`) pour représenter le texte
# - "classifier" : Utilise le modèle de classification logistique
pipe = Pipeline([("cleaner", cleaner),
                    ('vectorizer', bow_vector),
                    ('classifier', classifier)])

# Model generation
pipe.fit(x_train,y_train)

# Predicting with a test dataset
predicted = pipe.predict(x_test)

# Model Accuracy
print("Logistic Regression Accuracy:", accuracy_score(y_test, predicted))
print("Logistic Regression Precision:", precision_score(y_test, predicted))
print("Logistic Regression Recall:", recall_score(y_test, predicted))
print(confusion_matrix(y_test, predicted))

c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Mathieu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.7542222222222222
Logistic Regression Precision: 0.78210643722116
Logistic Regression Recall: 0.8532070224230837
[[3759 2735]
 [1689 9817]]
